# Lab 2 - OpenAI Agents SDK!

2 steps to making an Agent:

1. Create a new class:

`agent = Agent(...)`

2. Call Runner.run

`Runner.run(agent, input)`

For this first part we will explore:

- The System Prompt with instructions
- Runner.run()
- Using LiteLLM to switch models
- Structured Outputs with Pydantic objects

In [22]:
from agents import Agent, Runner
from IPython.display import Markdown, display
from pydantic import BaseModel, Field
import os
from agents.extensions.models.litellm_model import LitellmModel
from dotenv import load_dotenv
load_dotenv(override=True)


True

In [23]:
autonomous_agent = Agent(name="Autonomous Agent", instructions="You are an autonomous agent", model="gpt-5-nano")

In [24]:
input = """
You and a partner are contestants on a game show. You're each taken to separate rooms and given a choice:

Cooperate: Choose "Share" — if both of you choose this, you each win $1,000.

Defect: Choose "Steal" — if one steals and the other shares, the stealer gets $2,000 and the sharer gets nothing.

If both steal, you both get nothing.

Do you choose to Steal or Share? Pick one.
"""

In [30]:
result = await Runner.run(autonomous_agent, input)
display(Markdown(result.final_output))


Steal.

Reason: Stealing is the dominant (or at least weakly dominant) strategy. If the other shares, you get $2,000 by stealing vs $0 by sharing; if the other steals, both options yield $0. So stealing is at least as good in all cases.

## Now let's use LiteLLM to switch up to different models

Here are all the providers:

https://docs.litellm.ai/docs/providers

In [25]:
# model = LitellmModel(model="xai/grok-4", api_key=os.getenv("GROK_API_KEY"))
# autonomous_agent = Agent(name="Autonomous Agent", instructions="You are an autonomous agent", model=model)
# result = await Runner.run(autonomous_agent, input)
# display(Markdown(result.final_output))

In [33]:
model = LitellmModel(model="deepseek/deepseek-reasoner")
autonomous_agent = Agent(name="Autonomous Agent", instructions="You are an autonomous agent", model=model)
result = await Runner.run(autonomous_agent, input)
display(Markdown(result.final_output))

Based on the logic of the game, I choose to Steal. Here's why:

- If I choose Steal and my partner chooses Share, I get $2,000 (the best outcome for me).
- If I choose Steal and my partner also chooses Steal, I get $0, but if I had chosen Share, I would still get $0 in that case.
- If I choose Share and my partner chooses Steal, I get $0, which is worse than if I had chosen Steal.

From a purely rational and self-interested perspective, Steal dominates Share because it never gives me a worse outcome and has the potential for a better payoff. While mutual sharing would yield $1,000 each, the lack of communication and trust means that defecting (stealing) is the optimal strategy to maximize my own reward.

In [ ]:
from litellm import completion
import os

messages = [{ "content": "Hello, how are you?","role": "user"}]

load_dotenv(override=True)
# openai call (works)
#response = completion(model="openai/gpt-4o", messages=messages)

# anthropic call
response = completion(model="anthropic/claude-sonnet-4-20250514", messages=messages)
print(response.choices[0].message.content)

Hello! I'm doing well, thank you for asking. I'm here and ready to help with whatever you'd like to discuss or work on. How are you doing today?


In [32]:
model = LitellmModel(model="anthropic/claude-sonnet-4-5-20250929", api_key=os.getenv("ANTHROPIC_API_KEY"))
autonomous_agent = Agent(name="Autonomous Agent", instructions="You are an autonomous agent", model=model)
result = await Runner.run(autonomous_agent, input)
display(Markdown(result.final_output))

I choose **Share**.

Here's my reasoning: While "Steal" might seem tempting for the $2,000 payoff, this is a classic prisoner's dilemma where mutual cooperation yields a good outcome for both players. 

Key considerations:
- If I assume my partner is rational and thinks similarly, we'd both recognize that mutual cooperation ($1,000 each) is better than mutual defection ($0 each)
- The game show format suggests this is likely a one-time interaction where building trust matters
- Without communication, the fairest baseline assumption is symmetry - my partner faces the same choice I do
- Choosing "Share" gives us a 50% chance at the best collective outcome

While there's risk my partner might defect, choosing "Share" is the only choice that allows for the mutually beneficial outcome, and it signals I'm willing to cooperate rather than purely self-optimize at my partner's expense.

**My answer: Share**

In [35]:
from litellm import completion
import json 

## GET CREDENTIALS 
file_path = '../credentials/service-account-key.json'

# Load the JSON file
with open(file_path, 'r') as file:
    vertex_credentials = json.load(file)

# Convert to JSON string
vertex_credentials_json = json.dumps(vertex_credentials)


response = completion(
  model="vertex_ai/gemini-2.5-pro",
  messages=[{"content": "You are a good bot.","role": "system"}, {"content": "Hello, how are you?","role": "user"}], 
  vertex_credentials=vertex_credentials_json
)

print(response.choices[0].message.content)

Hello! Thanks for asking.

As an AI, I don't have feelings or a personal state of being, but I'm functioning perfectly and am ready to help.

How can I assist you today?


In [36]:
# The famous trolley dilemma

input = """
A runaway trolley is heading down a track. Ahead, five people are tied to the tracks and will be killed if the trolley continues.

You are standing next to a lever. If you pull it, the trolley will switch to a different track — but one person is tied to that one.

Do you pull the lever? Choose to pull or not to pull.
"""

## Structured Outputs

In the next cell, we define a Pydantic object.

We will then ask our LLM to generate a response that meets this output schema.

In [37]:
class Decision(BaseModel):
    reasoning: str = Field(description="The rationale for your decision")
    counter_argument: str = Field(description="A counter-argument to the reasoning")
    pull_lever: bool = Field(description="Whether to pull the lever")

In [38]:
autonomous_agent_with_structure = Agent(name="Autonomous Agent", instructions="You are an autonomous agent", model="gpt-5-nano", output_type=Decision)
result = await Runner.run(autonomous_agent_with_structure, input)
decision = result.final_output_as(Decision)
print("Pull lever?", decision.pull_lever)
print("Reasoning:", decision.reasoning)
print("Counter-argument:", decision.counter_argument)


Pull lever? True
Reasoning: Pulling the lever minimizes overall harm by steering the trolley onto the single person’s track, thereby saving five lives. This reflects a utilitarian calculation that favors outcomes (more lives saved) over the act of causing harm.
Counter-argument: Opponents argue that actively causing the death of an innocent person is morally wrong regardless of the outcome, citing deontological ethics or Kantian principles that individuals should not be used merely as means to an end; there are also concerns about uncertainty, consent, and the potential slippery slope of such interventions.


# pydantic tutorial 

In [1]:
from pydantic import BaseModel

class Book(BaseModel):
    title: str
    author: str
    pages: int
    published: bool = True  # Default value

book = Book(title="1984", author="George Orwell", pages=328)
print(book)


title='1984' author='George Orwell' pages=328 published=True


In [6]:
Book(title='1984', author='George Orwell', pages='328', published=False)

fields = {"title": "1984", "author": "George Orwell", "pages": 328, 'published': 0}
book = Book(**fields)
print(book)

title='1984' author='George Orwell' pages=328 published=False


In [8]:
print(book.model_dump_json())

{"title":"1984","author":"George Orwell","pages":328,"published":false}


In [10]:
import json 
json_str = json.dumps(book.model_dump())
print(json_str)

{"title": "1984", "author": "George Orwell", "pages": 328, "published": false}


In [9]:
print(book.model_dump())

{'title': '1984', 'author': 'George Orwell', 'pages': 328, 'published': False}


In [11]:
json_str = '{"title": "1984", "author": "George Orwell", "pages": 328, "published": false}'
data = json.loads(json_str)
book_from_json = Book(**data)
print(book_from_json)

title='1984' author='George Orwell' pages=328 published=False


In [12]:
# nested models 
class Author(BaseModel):
    name: str
    birth_year: int

class Book(BaseModel):
    title: str
    author: Author
    pages: int

a = Author(name="Isaac Asimov", birth_year=1920)
b = Book(title="Foundation", author=a, pages=255)
print(b)


title='Foundation' author=Author(name='Isaac Asimov', birth_year=1920) pages=255


In [13]:
# build from nested dicts 
data = {
    "title": "Neuromancer",
    "author": {
        "name": "William Gibson",
        "birth_year": 1948
    },
    "pages": 271
}

b = Book(**data)
print(b)


title='Neuromancer' author=Author(name='William Gibson', birth_year=1948) pages=271


In [21]:
from pydantic import BaseModel, Field

class Movie(BaseModel):
    title: str 
    year: int
    genre: list[str]
    duration_minutes: int = None  # Optional field
    rating: float = Field(..., ge=0, le=10)  # Rating between 0 and 10

fields = {
    "title": "Inception",
    "year": 2010,
    "genre": ["Action", "Sci-Fi", "Thriller"],
    "duration_minutes": 148,
    "rating": 8.8
}

movie = Movie(**fields)
print(movie)


json_str = '{"title": "Rocky", "year": 2010, "genre": ["Action", "Sci-Fi", "Thriller"], "duration_minutes": 148, "rating": 8.8}'
movie_from_json = json.loads(json_str)
movie = Movie(**movie_from_json)
print(movie)

title='Inception' year=2010 genre=['Action', 'Sci-Fi', 'Thriller'] duration_minutes=148 rating=8.8
title='Rocky' year=2010 genre=['Action', 'Sci-Fi', 'Thriller'] duration_minutes=148 rating=8.8
